[ Custom Model Class 정의 및 설계]
- 데이테셋 및 구현방법에 따른 모델 설계
- 부모클래스 : nn.Module
- 필수 오버라이딩 메서드 : 상속 받은 메서드 중 재정의하는 것
    *  _ _init_ _(self)   : 모델 구조/구성 요소 정의 
    * forward(self, data) : 전방향/순방향 학습 기능 메서드

[1] 모듈 로딩 <hr>

In [2]:
import torch                        # tensor, 내장함수 및 클래스 관련 모듈
import torch.nn as nn               # 인공신경망 관련 층, 함수 클래스 모듈
import torch.nn.functional as F     # 인공신경망 관련 층, AF, LF 함수 모듈

import pandas as pd                 # 데이터 로딩 및 전처리 관련 모듈 
import numpy as np                  # raw 데이터 즉, ndarray 데이터 관련 모듈


[2] 커스텀 모델 클래스 설계 <hr>

In [ ]:
## [2-1] 고정된 층과 퍼셉트론 구성 모델
## - 데 이  터 셋 : 피쳐 3 , 타겟 1 , 타겟 연속형
## - 학습    방법 : 지도학습 - 회귀
## - 구현알고리즘 : 인공신경망계열 DNN 
## -------------------------------------------------
##             입력       출력/퍼셉트론수    활성화함수
## - 입력층      3(피쳐)   20               ReLU
## - 은닉층      20       10                ReLU
## - 은닉층      10        5                ReLU
## - 출력층       5        1(타겟)            -
## 피쳐와 타겟을 제외하고 나머지는 사용자 지정 : 단, 초반에 퍼셉트론 수가 많은 것은 GOOD
## -------------------------------------------------
class CustomModel(nn.Module):
    # 인스턴스 초기화 및 생성 메서드
    def __init__(self):    # 인자를 필요로 하지 않음, self만 표시 
        # 부모 클래스 생성 
        super().__init__()
        # ANN 층 인스턴스 생성 : 자식 속성들 
        # ANN 층 객체 생성 이후, 객체 뒤에 () 붙이고 값 넣으면 
        # 해당 객체가 정의된 선형 변환 수행 
        self.in_layer=nn.Linear(3, 20)
        self.hd_layer1=nn.Linear(20, 10)
        self.hd_layer2=nn.Linear(10, 5)
        self.out_layer=nn.Linear(5, 1)

    # 순전파 학습 메서드 
    def forward(self, data):
        #- 입력층
        out=self.in_layer(data)      # data * weight + b
        out=F.relu(out)              # AF(data * weight + b) 결과물   (20개)

        #- 은닉층1
        out=self.hd_layer1(out)      # out * weight + b
        out=F.relu(out)              # AF(out * weight + b) 결과물    (10개)      

        #- 은닉층2
        out=self.hd_layer2(out)      # out * weight + b
        out=F.relu(out)              # AF(out * weight + b) 결과물    (5개)

        #- 출력층 : 회귀로 결과
        # 회귀, pytorch의 마지막 출력층에서는 활성화 함수가 필요 없음 
        out=self.out_layer(out)      # out * weight + b

        return out 
               

In [4]:
## - 모델 사용 -----------------------------------
## 모델 인스턴스 생성 
model=CustomModel()

## 모델 구조 확인
print(model)

CustomModel(
  (in_layer): Linear(in_features=3, out_features=20, bias=True)
  (hd_layer1): Linear(in_features=20, out_features=10, bias=True)
  (hd_layer2): Linear(in_features=10, out_features=5, bias=True)
  (out_layer): Linear(in_features=5, out_features=1, bias=True)
)


In [5]:
## 모델 동작 테스트
data=torch.FloatTensor([[10,5,8],[5,9,7]])
print(data)

## 모델에 데이터 전달
pre=model(data)

tensor([[10.,  5.,  8.],
        [ 5.,  9.,  7.]])


In [6]:
# 과대적합 발견 시 dropout

In [8]:
# 모델 파라미터들 
for name, params in model.named_parameters():
    print(name, params)

in_layer.weight Parameter containing:
tensor([[-0.0258, -0.1119, -0.0736],
        [ 0.0597, -0.3558,  0.4479],
        [-0.3439, -0.5404, -0.3694],
        [-0.0430,  0.0365, -0.0194],
        [-0.2102, -0.3077, -0.0117],
        [-0.0217,  0.1488, -0.4086],
        [-0.5481, -0.0407,  0.3343],
        [ 0.2275, -0.2880,  0.4706],
        [ 0.1174, -0.0893,  0.0159],
        [ 0.3644,  0.5571,  0.3325],
        [-0.1803,  0.1525,  0.5694],
        [ 0.4946,  0.3190, -0.5121],
        [-0.2192, -0.4605, -0.4885],
        [-0.3685, -0.1791, -0.3565],
        [-0.3671,  0.0355,  0.0028],
        [-0.4565, -0.1301,  0.2228],
        [-0.4318, -0.1810, -0.0076],
        [-0.3995,  0.0568,  0.3288],
        [ 0.3044, -0.0042, -0.3426],
        [ 0.5370,  0.2237,  0.2930]], requires_grad=True)
in_layer.bias Parameter containing:
tensor([-0.1852, -0.2793,  0.0297,  0.0366,  0.2555,  0.4488, -0.1493,  0.0811,
         0.3393, -0.3275, -0.2076,  0.1948, -0.1112, -0.5011, -0.4889, -0.3949,
     